In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from nltk.tokenize import TweetTokenizer
import datetime
#import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score
#from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

/home/joseph/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv('training.csv')
data_test = pd.read_csv('test.csv')

In [3]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/08/17,Credit reporting,NaN,Incorrect information on credit report,Account status,While I was informed by XXXX my credit score w...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NC,285XX,Older American,Consent provided,Web,03/08/17,Closed with explanation,Yes,No,2378505
1,07/28/17,Student loan,Federal student loan servicing,Problem with a credit reporting company's inve...,Problem with personal statement of dispute,Navient was supposed to send me documents regu...,NaN,"Navient Solutions, LLC.",MO,63301,NaN,Consent provided,Web,07/28/17,Closed with explanation,Yes,NaN,2589070
2,03/26/17,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,"XXXX XXXX, DE XXXX ( XXXX ) XXXX Requested On ...",Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",DE,197XX,NaN,Consent provided,Web,03/26/17,Closed with explanation,Yes,No,2404036
3,11/10/17,Mortgage,VA mortgage,Struggling to pay mortgage,NaN,We live in an area that was affected by Hurric...,Company believes it acted appropriately as aut...,"Pacific Union Financial, LLC",TX,77469,Servicemember,Consent provided,Web,11/10/17,Closed with explanation,No,NaN,2724813
4,02/27/17,Credit card,NaN,Billing disputes,NaN,"Hello team, I need your help. Please can you a...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",NaN,NaN,Older American,Consent provided,Web,03/02/17,Closed with explanation,Yes,No,2362090


# Product Categories

In [4]:
data['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    37463
Debt collection                                                                 24288
Mortgage                                                                        13436
Student loan                                                                     9831
Credit card or prepaid card                                                      8136
Credit reporting                                                                 6265
Checking or savings account                                                      4946
Credit card                                                                      3176
Bank account or service                                                          2576
Money transfer, virtual currency, or money service                               2174
Vehicle loan or lease                                                            2136
Consumer Loan                                         

# Subproduct - Student Loan

In [5]:
tf = data.loc[data.Product == 'Student loan' ]

In [6]:
tf.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
1,07/28/17,Student loan,Federal student loan servicing,Problem with a credit reporting company's inve...,Problem with personal statement of dispute,Navient was supposed to send me documents regu...,NaN,"Navient Solutions, LLC.",MO,63301,NaN,Consent provided,Web,07/28/17,Closed with explanation,Yes,NaN,2589070
6,01/18/17,Student loan,Federal student loan servicing,Can't repay my loan,Can't decrease my monthly payments,I 've been working with Navient Loans for over...,NaN,"Navient Solutions, LLC.",GA,312XX,NaN,Consent provided,Web,01/18/17,Closed with explanation,Yes,Yes,2295191
32,02/22/17,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Trouble with how payments are handled,Fedloans.org does not process payments in a ti...,NaN,AES/PHEAA,CA,950XX,NaN,Consent provided,Web,02/27/17,Closed with explanation,Yes,No,2353979
36,01/23/17,Student loan,Non-federal student loan,Dealing with my lender or servicer,Trouble with how payments are handled,I had gotten a loan from XXXX XXXX around XXXX...,NaN,"Navient Solutions, LLC.",CA,920XX,NaN,Consent provided,Web,01/23/17,Closed with explanation,Yes,No,2306568
42,01/26/17,Student loan,Non-federal student loan,Can't repay my loan,Can't decrease my monthly payments,I tried to refinance my student loans. Being t...,NaN,"Navient Solutions, LLC.",NY,112XX,NaN,Consent provided,Web,01/26/17,Closed with explanation,Yes,No,2313928


# Count across categories

In [7]:
tf['Sub-product'].value_counts()

Federal student loan servicing    6177
Non-federal student loan          1901
Private student loan              1753
Name: Sub-product, dtype: int64

In [8]:
tf['Consumer complaint narrative'].head()

1     Navient was supposed to send me documents regu...
6     I 've been working with Navient Loans for over...
32    Fedloans.org does not process payments in a ti...
36    I had gotten a loan from XXXX XXXX around XXXX...
42    I tried to refinance my student loans. Being t...
Name: Consumer complaint narrative, dtype: object

In [9]:
#tf = tf[['Sub-product','Consumer complaint narrative']]
tf1 = tf[['Consumer complaint narrative']]
tf2 = tf[['Sub-product']]

In [10]:
tf1.head()

,Consumer complaint narrative
1,Navient was supposed to send me documents regu...
6,I 've been working with Navient Loans for over...
32,Fedloans.org does not process payments in a ti...
36,I had gotten a loan from XXXX XXXX around XXXX...
42,I tried to refinance my student loans. Being t...


In [11]:
#data_test.head()

In [12]:
#data_test['Product'].value_counts()

In [11]:
tf_test = data.loc[data.Product == 'Student loan' ]

In [14]:
#tf_test.head()

In [15]:
#tf_test['Sub-product'].value_counts()

In [16]:
#tf_test['Consumer complaint narrative'].head()

In [12]:
tf_test1 = tf_test[['Consumer complaint narrative']]
tf_test2 = tf_test[['Sub-product']]

# Cleaning the data

In [13]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [14]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [15]:
tf1['clean_review']=clean_review(tf1.values)

/home/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
tf_test1['clean_review']=clean_review(tf_test1.values)

/home/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
tf_test1.head()

,Consumer complaint narrative,clean_review
1,Navient was supposed to send me documents regu...,navient wa supposed to send me document reguar...
6,I 've been working with Navient Loans for over...,i ve been working with navient loan for over a...
32,Fedloans.org does not process payments in a ti...,fedloans org doe not process payment in a time...
36,I had gotten a loan from XXXX XXXX around XXXX...,i had gotten a loan from xxxx xxxx around xxxx...
42,I tried to refinance my student loans. Being t...,i tried to refinance my student loan being tha...


In [18]:
tf1.head()

,Consumer complaint narrative,clean_review
1,Navient was supposed to send me documents regu...,navient wa supposed to send me document reguar...
6,I 've been working with Navient Loans for over...,i ve been working with navient loan for over a...
32,Fedloans.org does not process payments in a ti...,fedloans org doe not process payment in a time...
36,I had gotten a loan from XXXX XXXX around XXXX...,i had gotten a loan from xxxx xxxx around xxxx...
42,I tried to refinance my student loans. Being t...,i tried to refinance my student loan being tha...


In [19]:
tf2.head()

,Sub-product
1,Federal student loan servicing
6,Federal student loan servicing
32,Federal student loan servicing
36,Non-federal student loan
42,Non-federal student loan


In [20]:
X = tf1['clean_review']
#print(X)
Y = tf2

In [21]:
X_test = tf_test1['clean_review']
#print(X)
Y_test = tf_test2

# Total data

In [22]:
tokenizer = TweetTokenizer()

# Training data processing

In [23]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)
vectorizer.fit(X)
X_vectorized = vectorizer.transform(X)


# Applying ML

In [24]:
logreg = LogisticRegression()
ovr = OneVsRestClassifier(logreg)

In [25]:
%%time
ovr.fit(X_vectorized,Y)

CPU times: user 4.7 s, sys: 83.9 ms, total: 4.78 s
Wall time: 3.16 s


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [190]:
import numpy as np
scores = cross_val_score(ovr,X_vectorized,Y, scoring='accuracy', n_jobs=-1, cv=3)
print('Cross-validation mean accuracy {0:.2f}%, std {1:.2f}.'.format(np.mean(scores) * 100, np.std(scores) * 100))

Cross-validation mean accuracy 68.44%, std 0.39.


In [191]:
%%time
model = MultinomialNB()
#model.fit(train_vectorized, y)
scores =  cross_val_score(model, X_vectorized,Y, scoring='accuracy', n_jobs=-1, cv=3)
print('Cross-validation mean accuracy {0:.2f}%, std {1:.2f}.'.format(np.mean(scores) * 100, np.std(scores) * 100))

/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Cross-validation mean accuracy 62.83%, std 0.02.
CPU times: user 418 ms, sys: 121 ms, total: 539 ms
Wall time: 986 ms


In [26]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


# Corpus

In [27]:
all_words=' '.join(X)
all_words=word_tokenize(all_words)
#print(all_words)
dist=FreqDist(all_words)

num_unique_word=len(dist)
num_unique_word

16119

# Tokenize the words

In [28]:
tokenizer = Tokenizer(num_words=num_unique_word)

In [29]:
tokenizer.fit_on_texts(X)

In [30]:
x_train_tokens = tokenizer.texts_to_sequences(X)

In [31]:
x_test_tokens = tokenizer.texts_to_sequences(X_test)

# Padding and truncating data
The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. But if we take the longest sentence we waste a lot of memory. So, we truncate longer sentences and pad shorter sentences.

First we count the number of tokens in all the sequences in the data-set.

In [32]:
num_tokens = [len(tokens) for tokens in x_train_tokens ]
num_tokens = np.array(num_tokens)

The average number of tokens in a sequence is:

In [33]:
np.mean(num_tokens)

223.07232224595666

The maximum number of tokens in a sequence is:


In [34]:
np.max(num_tokens)

3221

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [35]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

642

In [204]:
#len(num_tokens)

This covers about 95% of the data-set.

In [36]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9546333028176177

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'.

In [37]:
pad = 'pre'

In [38]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [39]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [40]:
x_train_pad.shape

(9831, 642)

In [41]:
x_test_pad.shape

(9831, 642)

In [42]:
np.array(x_train_tokens[1])

array([   1,  186,   33,  270,   23,   26,    8,   16,   73,    7,   36,
         62,  187,    2,   49,   91,    5,   81,  414,   13,   78,   14,
         15,  237,    1,  455,   47,  165,   68,  777,   31,    5,  266,
         79,    2,   38,    1,  186,  987,   74,  685,    5,  717,   56,
          4,  237,  259,    2,  780,    6,  151,    5,   81,  901,    7,
         10,  185,  231,   14,  285,    2,  286,   14,   45,   20,  454,
          4,   79,   62,   14,   45,   20,  626,    2,  132,   23,   19,
          5,    1,   65,   20,  182,    4,   13,    5,   21,   22, 1249,
          6,   60,    5,   14,   45,  915,    2, 1169,    6,  317,    5,
        220])

In [43]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [50]:
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = array(Y)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y_encoded = onehot_encoder.fit_transform(integer_encoded)
print(Y_encoded)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
values_0 = array(Y_test)
#print(values)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values_0)
#print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y_encoded_test = onehot_encoder.fit_transform(integer_encoded)
print(Y_encoded_test)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# GLOVE
We would use pretrained embedding 'GLOVE', which is pretrained using billions of words, which could improve our accuracy compared to training our own embedding. Embeddings generally represent geometrical encodings of words based on how frequently appear together in a text corpus.

In [52]:
import gensim.models.keyedvectors as word2vec
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.100d.txt'


In [53]:

filename = 'glove.6B.100d.txt'
embeddings_index = dict()

            #Transfer the embedding weights into a dictionary by iterating through every line of the file.
f = open(filename)
for line in f:
                #split up line into an indexed array
    values = line.split()
   # print(values)
                #first index is word
    word = values[0]
                #store the rest of the values in the array as a new array
    coefs = np.asarray(values[1:], dtype='float32')
    #print(coefs)
    embeddings_index[word] = coefs #100 dimensions
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
  
        #We get the mean and standard deviation of the embedding weights so that we could maintain the 
        #same statistics for the rest of our own random generated weights. 
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()
        
   
        #We are going to set the embedding size to the pretrained dimension as we are replicating it.
        #the size will be Number of Words in Vocab X Embedding Size
embed_size = 100
embedding_matrix = np.random.normal(emb_mean, emb_std, (num_unique_word, embed_size))

   
        #With the newly created embedding matrix, we'll fill it up with the words 
        #that we have in both 
        #our own dictionary and loaded pretrained embedding. 
embeddedCount = 0
for word, i in tokenizer.word_index.items():
     
            #then we see if this word is in glove's dictionary, if yes, get the corresponding weights
        embedding_vector = embeddings_index.get(word)
   
            #and store inside the embedding matrix that we will train later on.
        if embedding_vector is not None: 
             embedding_matrix[i-1] = embedding_vector
        embeddedCount+=1
    #print(embeddedCount)
    #print('total embedded:',embeddedCount,'common words')
    

Loaded 400000 word vectors.


In [54]:
embedding_matrix

array([[-0.046539  ,  0.61966002,  0.56647003, ..., -0.37616   ,
        -0.032502  ,  0.80620003],
       [-0.18970001,  0.050024  ,  0.19084001, ..., -0.39804   ,
         0.47646999, -0.15983   ],
       [-0.1429    ,  1.06210005,  0.82244998, ...,  0.19328   ,
        -0.20184   ,  0.91416001],
       ...,
       [-0.31443228,  0.08975162,  0.39416614, ..., -0.25870429,
         0.18590435, -0.10240769],
       [-0.09377324,  0.01066256, -0.11949073, ..., -0.09724542,
        -0.10935151,  0.19399601],
       [ 0.22801   ,  0.77542001, -0.038877  , ...,  0.22254001,
         0.16836999,  0.48008999]])

In [55]:
embedding_matrix.shape

(16119, 100)

Embedding matrix from pretrained model would be used as embedding for the LSTM model

In [56]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [61]:
inp = Input(shape=(max_tokens, ))

In [62]:
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)

In [63]:
x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)

In [64]:
x = GlobalMaxPool1D()(x)

In [65]:
x = Dropout(0.10)(x)

In [66]:
x = Dense(50, activation="relu")(x)

In [67]:
x = Dropout(0.10)(x)

In [68]:
x = Dense(3, activation="softmax")(x)

In [69]:
model = Model(inputs=inp, outputs=x)

In [70]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [71]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 642)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 642, 100)          1611900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 642, 120)          77280     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 120)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6050      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [77]:
%%time

batch_size = 32
epochs = 10
model.fit(x_train_pad, Y_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 8847 samples, validate on 984 samples
Epoch 1/10
8847/8847 [==============================] - 214s 24ms/step - loss: 0.7315 - acc: 0.7001 - val_loss: 0.7567 - val_acc: 0.6819
Epoch 2/10
8847/8847 [==============================] - 215s 24ms/step - loss: 0.7003 - acc: 0.7103 - val_loss: 0.7469 - val_acc: 0.6839
Epoch 3/10
8847/8847 [==============================] - 217s 24ms/step - loss: 0.6906 - acc: 0.7129 - val_loss: 0.7533 - val_acc: 0.6850
Epoch 4/10
8847/8847 [==============================] - 215s 24ms/step - loss: 0.6627 - acc: 0.7222 - val_loss: 0.7379 - val_acc: 0.6911
Epoch 5/10
8847/8847 [==============================] - 214s 24ms/step - loss: 0.6466 - acc: 0.7332 - val_loss: 0.7806 - val_acc: 0.6982
Epoch 6/10
8847/8847 [==============================] - 214s 24ms/step - loss: 0.6270 - acc: 0.7390 - val_loss: 0.7411 - val_acc: 0.6890
Epoch 7/10
8847/8847 [==============================] - 216s 24ms/step - loss: 0.6064 - acc: 0.7547 - val_loss: 0.7417 - val_acc: 0

In [79]:
%%time
result = model.evaluate(x_test_pad, Y_encoded_test)

9831/9831 [==============================] - 58s 6ms/step
CPU times: user 3min 19s, sys: 16.3 s, total: 3min 36s
Wall time: 58.2 s


In [80]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 83.11%


# without pretrained model

In [82]:
inp = Input(shape=(max_tokens, ))

In [83]:
embed_size = 128
x = Embedding(num_unique_word, embed_size)(inp)

In [84]:
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)

In [85]:


x = GlobalMaxPool1D()(x)



In [86]:
x = Dropout(0.1)(x)

In [87]:
x = Dense(50, activation="relu")(x)

In [88]:
x = Dropout(0.1)(x)

In [89]:
x = Dense(3, activation="sigmoid")(x)

In [90]:
model1 = Model(inputs=inp, outputs=x)

In [91]:
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [92]:
batch_size = 32
epochs = 10
model1.fit(x_train_pad, Y_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 8847 samples, validate on 984 samples
Epoch 1/10
8847/8847 [==============================] - 125s 14ms/step - loss: 0.9286 - acc: 0.6234 - val_loss: 0.8879 - val_acc: 0.6209
Epoch 2/10
8847/8847 [==============================] - 127s 14ms/step - loss: 0.7585 - acc: 0.6834 - val_loss: 0.6984 - val_acc: 0.7043
Epoch 3/10
8847/8847 [==============================] - 127s 14ms/step - loss: 0.6398 - acc: 0.7389 - val_loss: 0.7046 - val_acc: 0.6931
Epoch 4/10
8847/8847 [==============================] - 128s 14ms/step - loss: 0.5484 - acc: 0.7780 - val_loss: 0.7295 - val_acc: 0.6921
Epoch 5/10
8847/8847 [==============================] - 128s 14ms/step - loss: 0.4398 - acc: 0.8297 - val_loss: 0.7939 - val_acc: 0.7083
Epoch 6/10
8847/8847 [==============================] - 132s 15ms/step - loss: 0.3607 - acc: 0.8637 - val_loss: 0.8321 - val_acc: 0.7022
Epoch 7/10
8847/8847 [==============================] - 128s 14ms/step - loss: 0.2625 - acc: 0.9071 - val_loss: 0.9636 - val_acc: 0

In [93]:
%%time
result = model1.evaluate(x_test_pad, Y_encoded_test)

9831/9831 [==============================] - 33s 3ms/step
CPU times: user 1min 45s, sys: 8.72 s, total: 1min 54s
Wall time: 32.6 s


In [94]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 95.21%
